In [1]:
import os 
import json

from PIL import Image
from config import *
from tqdm import tqdm

In [2]:
dataset_path  = "data/"

raw_data_path = f"{dataset_path}/raw_data"

In [3]:
images_path = "images"
label_path = "labels"

train_path = "train"
val_path = "val"
test_path = "test"

defect_images_path = "defect_Images"
normal_images_path = "normal_Images"
annotations = "Annotations/anno_train.json"

In [4]:
raw_data_path = [f'{raw_data_path}/train1', f'{raw_data_path}/train2']


In [5]:
yolo_label_path = f"{dataset_path}/yolo/labels/"
yolo_image_path = f"{dataset_path}/yolo/images/"

In [6]:
os.makedirs(yolo_image_path, exist_ok=True)
os.makedirs(yolo_label_path, exist_ok=True)

In [7]:
def read_raw_data(path:str, test:bool=False):
    with open(f"{path}/{annotations}", "rb") as f:
        label_json = json.load(f)
    # defect_images_files = os.listdir(f"{path}/{defect_images_path}")
    normall_images_files = os.listdir(f"{path}/{normal_images_path}")

    # tqdm()
    print(len(label_json))
    not_exists = 0
    for item in tqdm(label_json):
        name = item["name"]
        bbox = item["bbox"]
        
        if  not os.path.exists(f"{path}/{defect_images_path}/{name}"):
            # print(f"{path}/{defect_images_path}/{name}")
            not_exists+=1
            continue
        with Image.open(f"{path}/{defect_images_path}/{name}") as img:
            original_size = img.size
            
            
            
            label_file_path = f"{yolo_label_path}/{name.split(".")[0]}.txt" 
            img_file_path = f"{yolo_image_path}/{name.split(".")[0]}.jpg"
            
            with open(label_file_path, "a+") as label_file:
                x_center = (bbox[0] + bbox[2]) / 2
                y_center = (bbox[1] + bbox[3]) / 2
                width = bbox[2] - bbox[0]
                height = bbox[3] - bbox[1]
                
                x_center = x_center / original_size[0]
                y_center = y_center / original_size[1]
                width = width / original_size[0]
                height = height / original_size[1]
                label_file.write(str(0) + " " + str(x_center) + " " + str(y_center) + " " + str(width) + " " + str(height) + '\n')
            if not  os.path.exists(img_file_path):
                # os.remove(img_file_path)
                resized_img = img.resize((IMAGE_SIZE, IMAGE_SIZE))
                resized_img.save(img_file_path)
    
    
    print((not_exists))        
    for nomal_item in tqdm(normall_images_files):
         with Image.open(f"{path}/{normal_images_path}/{nomal_item}") as img:
            resized_img = img.resize((IMAGE_SIZE,IMAGE_SIZE))
            label_file_path = f"{yolo_label_path}/{nomal_item.split(".")[0]}.txt" 
            img_file_path = f"{yolo_image_path}/{nomal_item.split(".")[0]}.jpg"
            
            if not os.path.exists(label_file_path):
                with open(label_file_path, "a+") as label_file:
                    pass
            if not os.path.exists(img_file_path):
                with Image.open(f"{path}/{normal_images_path}/{nomal_item}") as img:
                    resized_img = img.resize((IMAGE_SIZE, IMAGE_SIZE))
                    resized_img.save(img_file_path)
           
                

In [8]:
read_raw_data(raw_data_path[1])

1795


100%|██████████| 1795/1795 [00:25<00:00, 69.40it/s]


0


100%|██████████| 1125/1125 [00:51<00:00, 21.82it/s]


In [9]:
read_raw_data(raw_data_path[0])

9273


100%|██████████| 9273/9273 [01:50<00:00, 83.72it/s] 


1674


100%|██████████| 2518/2518 [01:49<00:00, 23.07it/s]


In [10]:
from pathlib import Path
import random
import shutil


image_dir = Path('data/yolo/images')
label_dir = Path('data/yolo/labels')

# Create output directories
for split in ['train', 'val', 'test']:
    (image_dir / split).mkdir(exist_ok=True)
    (label_dir / split).mkdir(exist_ok=True)

# Get all image files (assuming .jpg or .png)
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]
random.shuffle(image_files)

# Define split sizes
total = len(image_files)
train_split = int(total * 0.7)
val_split = int(total * 0.2)
test_split = total - train_split - val_split

# Assign file lists
train_files = image_files[:train_split]
val_files = image_files[train_split:train_split + val_split]
test_files = image_files[train_split + val_split:]

# Function to move files
def move_files(files, dest):
    for f in files:
        base_name = Path(f).stem
        image_src = image_dir / f
        label_src = label_dir / f"{base_name}.txt"
        
        image_dst = image_dir / dest / f
        label_dst = label_dir / dest / f"{base_name}.txt"
        
        if image_src.exists():
            shutil.copy(str(image_src), str(image_dst))
        if label_src.exists():
            shutil.copy(str(label_src), str(label_dst))

# Move files
move_files(train_files, 'train')
move_files(val_files, 'val')
move_files(test_files, 'test')

print("✅ Dataset split completed.")

✅ Dataset split completed.
